In [ ]:
%matplotlib inline
import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
%matplotlib widget
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy import stats

# Some useful utilities
from mcmc_utils_and_plot import scatter_matrix, build_cov_mat, lognormpdf, plot_bivariate_gauss, eval_func_on_grid, sub_sample_data

def compose(*functions):
    "Compose a list of functions"
    return functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [ ]:
def odeFuncI(t,x):
    N = 1000
    S = x[0]
    I = x[1]
    R = x[2]
    beta = 0.2
    r = 0.6
    delta = 0.15
    # theta = np.array([beta, r, delta]) # true parameter settings
    y0 = delta*N - delta*S - beta*I*S  # del S
    y1 = beta*I*S - (r+delta)*I        # del I
    y2 = r*I - delta*R                 # del R
    return y0,y1,y2

In [ ]:
def solverFuncI():
    timeVec = np.linspace(0,6,61)
    S_0 = 900
    I_0 = 100
    R_0 = 0
    x_0 = np.array([S_0, I_0, R_0])
    soln = scipy.integrate.solve_ivp(odeFuncI,[0,6],x_0,t_eval=timeVec,method='RK45')
    return soln

In [ ]:
solnI = solverFuncI()
data1 = np.zeros(61)
stdNoise = 50 
for ii in range (len(data1)):
    data1[ii]=solnI.y[1,ii]+stdNoise*np.random.randn()

In [ ]:
plt.figure(num='Identifiable Version',figsize=(10,8))
plt.plot(solnI.t,solnI.y[0,:],'y',linewidth=4,label='Susceptible')
plt.plot(solnI.t,solnI.y[1,:],'m',linewidth=4,label='Infected')
plt.plot(solnI.t,solnI.y[2,:],'c',linewidth=4,label='Recovered')
plt.legend()
plt.xlabel("Time")
plt.ylabel("Population")
plt.show()

In [ ]:
plt.figure(num='Data',figsize=(10,8))
plt.plot(solnI.t,solnI.y[1,:],'m',linewidth=4,label='Infected')
plt.scatter(solnI.t,data1, c='r', marker='X',s=33,label='Data')
plt.legend()
plt.xlabel("Time")
plt.ylabel("Population")
plt.show()

In [ ]:
def odeFuncU(t,x):
    N = 1000
    S = x[0]
    I = x[1]
    R = x[2]
    gamma = 0.2
    kappa = 0.1
    r = 0.6
    delta = 0.15
    # theta = np.array([gamma, kappa, r, delta]) # true parameter settings
    y0 = delta*N - delta*S - gamma*kappa*I*S  # del S
    y1 = gamma*kappa*I*S - (r+delta)*I        # del I
    y2 = r*I - delta*R                 # del R
    return y0,y1,y2

In [ ]:
def solverFuncU():
    timeVec = np.linspace(0,6,61)
    S_0 = 900
    I_0 = 100
    R_0 = 0
    x_0 = np.array([S_0, I_0, R_0])
    soln = scipy.integrate.solve_ivp(odeFuncU,[0,6],x_0,t_eval=timeVec,method='RK45')
    return soln

In [ ]:
solnU = solverFuncU()
data2 = np.zeros([61,1])
stdNoise = 50
for ii in range (0,len(data2)):
    data2[ii]=solnU.y[1,ii]+stdNoise*np.random.randn()

In [ ]:
plt.figure(num='Undentifiable Version',figsize=(10,8))
plt.plot(solnU.t,solnU.y[0,:],'y',linewidth=4,label='Susceptible')
plt.plot(solnU.t,solnU.y[1,:],'m',linewidth=4,label='Infected')
plt.plot(solnU.t,solnU.y[2,:],'c',linewidth=4,label='Recovered')
plt.legend()
plt.xlabel("Time")
plt.ylabel("Population")
plt.show()

In [ ]:
plt.figure(num='Data',figsize=(10,8))
plt.plot(solnU.t,solnU.y[1,:],'m',linewidth=4,label='Infected')
plt.scatter(solnU.t,data2, c='r', marker='X',s=33,label='Data')
plt.legend()
plt.xlabel("Time")
plt.ylabel("Population")
plt.show()

In [ ]:
    beta = 0.2
    r = 0.6
    delta = 0.15
    theta_t = np.array([beta, r, delta]) # true parameter settings
    #theta = np.random.randn(3)
    print(theta_t)
    gamma = 0.2
    kappa = 0.1
    theta_tU = np.array([gamma, kappa,r, delta]) # true parameter settings
    print(theta_tU)

In [ ]:
def log_likelihoodI(thetain,data):
    timeVec = np.linspace(0,6,61)
    beta = thetain[0]
    r = thetain[1]
    delta = thetain[2]
    S_0 = 900
    I_0 = 100
    R_0 = 0
    N = 1000
    x_0 = np.array([S_0, I_0, R_0])
    soln = scipy.integrate.solve_ivp(odeFuncI, [0,6], y0=x_0, method='RK45', t_eval=timeVec)

#     print(np.round(np.sum(soln.y,axis=0)))
#     if np.any(np.round(np.sum(soln.y,axis=0))!=N):
#         return -np.inf
    if thetain[0] >= 0 and thetain[1] >= 0 and thetain[2] >= 0:
        std = 50
#         pre_exp_term = 1/(np.sqrt(2*np.pi)*std)
#         exp_term = -0.5*np.square((data.T-soln.y[1])/std)
#         f = np.log(pre_exp_term)+exp_term
        f = np.zeros(len(soln.y[1,:]))
        for ii in range(len(soln.y[1,:])):
             f[ii] = np.log(scipy.stats.norm.pdf(data[ii],soln.y[1,ii], std))
        f_theta0 = (np.log(scipy.stats.norm.pdf(thetain[0],0,1)))
        f_theta1 = (np.log(scipy.stats.norm.pdf(thetain[1],0,1)))
        f_theta2 = (np.log(scipy.stats.norm.pdf(thetain[2],0,1)))
        f_theta = np.sum(np.array([f_theta0, f_theta1, f_theta2]))
        out=np.sum(f) + f_theta
        
    else:
        return -np.inf
        

    return out

In [ ]:
logpostI = lambda params: log_likelihoodI(params, data1)

In [ ]:
def multiVarLogPDF(x,y,cov):
    x_mu = x-y
    logpdf = np.log((1/(np.sqrt((2*np.pi)**2*np.linalg.det(cov))))*np.exp(-0.5*np.dot((x_mu).T,np.dot(np.linalg.inv(cov),x_mu))))
    return logpdf

In [ ]:
class DelayedRejectionAdaptiveMetropolis:
    
    def __init__(self, logpdf, cov, t0=100, freq=10, sd=None, max_samples=10000, eps=1e-7): 
        """The class constructor, parameters are documented below"""
        self.logpdf = logpdf # callable (param) -> logpdf defining the logpdf
        self.cov = cov # initial covariance
        self.cov_chol = np.linalg.cholesky(cov) # sqrt of the covariance
        self.dim = cov.shape[0] # number of parameters
        self.t0 = t0 # time to start adapting
        self.freq = freq # frequency of covariance updates (should be an integer > 0)
        if sd == None:
            self.sd = (2.4**2) / self.dim
        else:
            self.sd = sd # scale for the covariance                    
        self.max_samples = max_samples # maximum number of samples
        self.eps = eps # nugget for ensuring positive definite
        self.num_samples = 0 # number of samples generated
        self.samples = np.zeros((max_samples, self.dim)) # store the samples
        self.logpdf_vals = np.zeros((max_samples))
        
        self.sample_mean = self.samples[0,:]
        
    def sample(self, initial_sample, num_samples):
    
        assert num_samples <= self.max_samples, "Requesting more samples than space is allocated for"
        
        self.samples[0, :] = initial_sample
        self.logpdf_vals[0] = self.logpdf(initial_sample)
        
        accept = 1
        for ii in range(1, num_samples):
            
            # propose
            y = self.samples[ii-1, :] + np.dot(self.cov_chol, np.random.randn(self.dim))
            y_logpdf = self.logpdf(y)
            
            # compute accept-reject probability, using the fact that we have a symmetric proposal
            a = np.exp(y_logpdf - self.logpdf_vals[ii-1])
            a = min(a, 1)
    
            u = np.random.rand()
        
            if u < a: #accept
                self.samples[ii, :] = y
                self.logpdf_vals[ii] = y_logpdf
                accept += 1
            else:
                # propose
                self.num_samples += 1
                chollev2 = np.linalg.cholesky(0.5*self.cov) # sqrt of the covariance
                y2 = self.samples[ii-1, :] + np.dot(chollev2, np.random.randn(self.dim))
                y_logpdf2 = self.logpdf(y2)
                
                a1y2y1 = np.exp(y_logpdf-y_logpdf2)
                a1xy1 = np.exp(y_logpdf - self.logpdf_vals[ii-1])
                
                q1 = multiVarLogPDF(y,y2,self.cov)
                q2 = multiVarLogPDF(y,self.samples[ii-1, :],self.cov)
                
                a2 = np.exp((y_logpdf2+np.log(1-a1y2y1))-(self.logpdf_vals[ii-1]+np.log(1-a1xy1)) + q1 - q2)
                a2 = min(a2,1)
                
                u2 = np.random.rand()

                if u2 < a2: # accept
                    self.samples[ii, :] = y2
                    self.logpdf_vals[ii] = y_logpdf2
                    accept += 1
                else:
                    self.samples[ii, :] = self.samples[ii-1, :]
                    self.logpdf_vals[ii] = self.logpdf_vals[ii-1]
                    
            self.num_samples += 1
             
            
            # adapt covariance if its time
            if ii > self.t0 and ii % self.freq == 0:
                sampleMeanNew = (1/(ii+1))*self.samples[ii, :]+(ii/(ii+1))*self.sample_mean
                covUpdate     = self.eps*np.eye(self.dim) + ii*np.dot(self.sample_mean,self.sample_mean) - \
                                (ii+1)*np.dot(sampleMeanNew,sampleMeanNew) + \
                                 np.dot(self.samples[ii, :],self.samples[ii, :])
                self.cov = (ii-1)*self.cov/ii + self.sd*covUpdate/ii 
                self.cov_chol = np.linalg.cholesky(self.cov)
                self.sample_mean = sampleMeanNew           
            if ii % 1000 == 0:
                print(f"Finished sample {ii}, acceptance ratio = {accept / self.num_samples}")
                
        return self.samples, accept / float (self.num_samples)

In [ ]:
def laplace_approx(initial_guess, logpost):
    """Perform the laplace approximation, 
        returning the MAP point and an approximation of the covariance
        
    Inputs
    ------
    initial_guess: (nparam, ) array of initial parameters
    logpost: function (param) -> log posterior
    
    Ouputs
    ------
    map_point: (nparam, ) MAP of the posterior
    cov_approx: (nparam, nparam), covariance matrix for Gaussian fit at MAP
    """
    def neg_post(x):
        """Negative posteror because optimizer is a minimizer"""
        return -logpost(x)
    
    # Gradient free method to obtain optimum
    res = scipy.optimize.minimize(neg_post, initial_guess, method='Nelder-Mead') 
    # Gradient method which also approximates the inverse of the hessian
    res = scipy.optimize.minimize(neg_post, res.x)

    map_point = res.x
    cov_approx = res.hess_inv
    return map_point, cov_approx

In [ ]:
num_samples = 10000
dim=3
guess = np.random.randn((dim)) # random guess
map_point, cov_laplace = laplace_approx(guess, logpostI)
initial_sample = map_point
covin = cov_laplace
print(cov_laplace)
print(map_point)

data = data1
dram = DelayedRejectionAdaptiveMetropolis(logpostI, covin, freq=5, t0=300, sd = None, eps = 1e-7, max_samples=num_samples)

samples, ar = dram.sample(initial_sample,num_samples)
print(samples)



In [ ]:
def sub_sample_data(samples, frac_burn=0.2, frac_use=0.7):
    """Subsample data by burning off the front fraction and using another fraction

    Inputs
    ------
    samples: (N, d) array of samples
    frac_burn: fraction < 1, percentage of samples from the front to ignore
    frac_use: percentage of samples to use after burning, uniformly spaced
    """
    nsamples = samples.shape[0]
    inds = np.arange(nsamples, dtype=np.int)
    start = int(frac_burn * nsamples)
    inds = inds[start:]
    nsamples = nsamples - start
    step = int(nsamples / (nsamples * frac_use))
    inds2 = np.arange(0, nsamples, step)
    inds = inds[inds2]
    return samples[inds, :]

In [ ]:
samples = sub_sample_data(samples, frac_burn=0.2, frac_use=0.7)
fig, axs, gs = scatter_matrix([samples_sub], hist_plot=False, gamma=0.2, labels=[r'$\theta_1$', r'$\theta_2$', r'$\theta_3$'])
fig.set_size_inches(10,10)
plt.subplots_adjust(left=0.01, right=0.99, wspace=0.2, hspace=0.2)

In [ ]:
def autocorrelation(samples, maxlag=100, step=1):
    """Compute the correlation of a set of samples
    
    Inputs
    ------
    samples: (N, d)
    maxlag: maximum distance to compute the correlation for
    step: step between distances from 0 to maxlag for which to compute teh correlations
    """
    
    # Get the shapes
    ndim = samples.shape[1]
    nsamples = samples.shape[0]    
    
    # Compute the mean
    mean = np.mean(samples, axis=0)
    
    # Compute the denominator, which is variance
    denominator = np.zeros((ndim))
    for ii in range(nsamples):
        denominator = denominator + (samples[ii, :] - mean)**2
    
    lags = np.arange(0, maxlag, step)
    autos = np.zeros((len(lags), ndim))
    for zz, lag in enumerate(lags):
        autos[zz, :] = np.zeros((ndim))
        # compute the covariance between all samples *lag apart*
        for ii in range(nsamples - lag):
            autos[zz,:] = autos[zz, :] + (samples[ii,:]-mean)*(samples[ii + lag,:] -mean)
        autos[zz, :] = autos[zz, :]/denominator
    return lags, autos

In [ ]:
maxlag=500
step=1
lags, autolag = autocorrelation(samples, maxlag=maxlag,step=step)
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].plot(lags, autolag[:, 0],'-o')
axs[0].set_xlabel('lag')
axs[0].set_ylabel('autocorrelation dimension 1')
axs[1].plot(lags, autolag[:, 1],'-o')
axs[1].set_xlabel('lag')
axs[1].set_ylabel('autocorrelation dimension 2')
axs[2].plot(lags, autolag[:, 2],'-o')
axs[2].set_xlabel('lag')
axs[2].set_ylabel('autocorrelation dimension 3')
plt.show()

IAC1 = 1+2*np.sum(autolag[:,0])
print(IAC1)
IAC2 = 1+2*np.sum(autolag[:,1])
print(IAC2)
IAC2 = 1+2*np.sum(autolag[:,2])
print(IAC2)

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(10,5))
axs[0].plot(samples[:, 0], '-k')
axs[0].set_ylabel(r'$x_1$', fontsize=14)
axs[1].plot(samples[:, 1], '-k')
axs[1].set_ylabel(r'$x_2$', fontsize=14)
axs[1].set_xlabel('Sample Number', fontsize=14)
axs[2].plot(samples[:, 2], '-k')
axs[2].set_ylabel(r'$x_2$', fontsize=14)
axs[2].set_xlabel('Sample Number', fontsize=14)
#axs[1].set_xlim([40000, 50000])

In [ ]:
def log_likelihoodU(thetain,data):
    timeVec = np.linspace(0,6,61)
    gamma = thetain[0]
    kappa = thetain[1]
    r = thetain[2]
    delta = thetain[3]
    S_0 = 900
    I_0 = 100
    R_0 = 0
    N = 1000
    x_0 = np.array([S_0, I_0, R_0])
    soln = scipy.integrate.solve_ivp(odeFuncU, [0,6], y0=x_0, method='RK45', t_eval=timeVec)

#     print(np.round(np.sum(soln.y,axis=0)))
#     if np.any(np.round(np.sum(soln.y,axis=0))!=N):
#         return -np.inf
    if thetain[0] >= 0 and thetain[1] >= 0 and thetain[2] >= 0 and thetain[3] >= 0:
        std = 50
#         pre_exp_term = 1/(np.sqrt(2*np.pi)*std)
#         exp_term = -0.5*np.square((data.T-soln.y[1])/std)
#         f = np.log(pre_exp_term)+exp_term
        f = np.zeros(len(soln.y[1,:]))
        for ii in range(len(soln.y[1,:])):
             f[ii] = np.log(scipy.stats.norm.pdf(data[ii], soln.y[1,ii],std))
        f_theta0 = (np.log(scipy.stats.norm.pdf(thetain[0],0,1)))
        f_theta1 = (np.log(scipy.stats.norm.pdf(thetain[1],0,1)))
        f_theta2 = (np.log(scipy.stats.norm.pdf(thetain[2],0,1)))
        f_theta3 = (np.log(scipy.stats.norm.pdf(thetain[3],0,1)))
        f_theta = np.sum(np.array([f_theta0, f_theta1, f_theta2, f_theta3]))
        out=np.sum(f) + f_theta
        
    else:
        return -np.inf
        

    return out

In [ ]:
logpostU = lambda params: log_likelihoodU(params, data2)

In [ ]:
num_samples = 5000
dim=4
guess = np.random.randn((dim)) # random guess
map_point, cov_laplace = laplace_approx(guess, logpostU)
initial_sample = map_point
covin = cov_laplace
print(cov_laplace)
print(map_point)

dram = DelayedRejectionAdaptiveMetropolis(logpostU, covin, freq=5, t0=300, sd = None, eps = 1e-7, max_samples=num_samples)


samples, ar = dram.sample(theta_tU,num_samples)
print(samples)




In [ ]:
samples = sub_sample_data(samples, frac_burn=0.2, frac_use=0.7)
fig, axs, gs = scatter_matrix([samples_sub], hist_plot=False, gamma=0.2)
fig.set_size_inches(10,10)
plt.subplots_adjust(left=0.01, right=0.99, wspace=0.2, hspace=0.2)

In [ ]:
maxlag=500
step=1
lags, autolag = autocorrelation(samples, maxlag=maxlag,step=step)
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].plot(lags, autolag[:, 0],'-o')
axs[0].set_xlabel('lag')
axs[0].set_ylabel('autocorrelation dimension 1')
axs[1].plot(lags, autolag[:, 1],'-o')
axs[1].set_xlabel('lag')
axs[1].set_ylabel('autocorrelation dimension 2')
axs[2].plot(lags, autolag[:, 2],'-o')
axs[2].set_xlabel('lag')
axs[2].set_ylabel('autocorrelation dimension 3')
axs[3].plot(lags, autolag[:, 3],'-o')
axs[3].set_xlabel('lag')
axs[3].set_ylabel('autocorrelation dimension 4')

plt.show()

IAC1 = 1+2*np.sum(autolag[:,0])
print(IAC1)
IAC2 = 1+2*np.sum(autolag[:,1])
print(IAC2)
IAC2 = 1+2*np.sum(autolag[:,2])
print(IAC2)
IAC3 = 1+2*np.sum(autolag[:,3])
print(IAC3)

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(10,10))
axs[0].plot(samples[:, 0], '-k')
axs[0].set_ylabel(r'$x_1$', fontsize=14)
axs[1].plot(samples[:, 1], '-k')
axs[1].set_ylabel(r'$x_2$', fontsize=14)
axs[1].set_xlabel('Sample Number', fontsize=14)
axs[2].plot(samples[:, 2], '-k')
axs[2].set_ylabel(r'$x_2$', fontsize=14)
axs[2].set_xlabel('Sample Number', fontsize=14)
axs[3].plot(samples[:, 3], '-k')
axs[3].set_ylabel(r'$x_2$', fontsize=14)
axs[3].set_xlabel('Sample Number', fontsize=14)
#axs[1].set_xlim([40000, 50000])